# HEP Jet assignment project - Data analysis and particle finding script

## Import essential packages.
---
* We will use [uproot](https://github.com/scikit-hep/uproot) packages to parse our .root file.
* The content of function `particle properties` and `jet properties` is defined in `particle_properties.py` and `jet_properties.py`.

In [2]:
import uproot
import pandas as pd 
import numpy as np 
from particle_properties_uproot import particle_properties  #import particle properties helper function from particle_properties.py
from jet_properties_uproot import jet_properties  #import jet properties helper function from jet_properties.py
import h5py

## Loading data, determine parameters, and assign variable
---

In [3]:
data  = uproot.open('./tag_1_delphes_events.root')['Delphes']
#data.show()

particle = particle_properties(data)
jet = jet_properties(data)

Length = len(particle.event)
test_length = 10

PID_W_plus = 24 
PID_W_minus = -24
PID_DOWN = 1
PID_DOWN_VAR = -1
PID_UP = 2
PID_UP_BAR = -2
PID_STRANGE = 3
PID_STRANGE_BAR = -3
PID_CHARM = 4
PID_CHARM_BAR = -4
PID_BOTTOM = 5
PID_BOTTOM_BAR = -5
PID_TOP = 6
PID_TOP_BAR = -6

top_idx = np.zeros(len(particle.event))
top_daughter_idx_1 = np.zeros(len(particle.event))
top_daughter_pid_1 = np.zeros(len(particle.event))
top_daughter_idx_2 = np.zeros(len(particle.event))
top_daughter_pid_2 = np.zeros(len(particle.event))

top_bar_idx = np.zeros(len(particle.event))
top_bar_daughter_idx_1 = np.zeros(len(particle.event))
top_bar_daughter_pid_1 = np.zeros(len(particle.event))
top_bar_daughter_idx_2 = np.zeros(len(particle.event))
top_bar_daughter_pid_2 = np.zeros(len(particle.event))



## Event selection 
---
1. Must contain:
    * At least 2 b tagged jet.
    * At least 6 jet exists.
    * For each jet, require |$\eta$| < 2.4 and $P_{T}$ > 20GeV. 

In [5]:
#Generate maker for each stage(event selection and jet selection.)
marker_event = []
marker_jet = []
marker_bjet = []

for i in range(len(particle.event)):
    marker_event.append(0)
    marker_jet.append(np.zeros([len(jet.pt[i])]))
    marker_bjet.append(np.zeros([len(jet.pt[i])]))


marker_event = np.asanyarray(marker_event)
marker_jet = np.asanyarray(marker_jet)
marker_bjet = np.asanyarray(marker_bjet)

print(type(marker_event), type(marker_jet))
print(marker_event.shape, marker_jet.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(10000,) (10000,)


In [6]:
#Mark which event pass the selection
# print("+-----------------------------------------------------------------------------------------------------+")
# print("Start event selection.")
# for i in range(len(particle.event)):
#     num_of_eta_in_range = np.sum(jet.eta[i] < 2.4 ) 
#     num_of_jet = len(jet.pt[i])
#     num_of_btagged = np.sum(jet.btag[i] == 1)
#     if num_of_eta_in_range >= 6 and num_of_jet >=6 and num_of_btagged >= 2: 
#         marker_event[i] = 1
#     else :
#         pass
# print("Event selection doen.")
# print("+-----------------------------------------------------------------------------------------------------+")

#Mark which jet in each event pass the selection.
print("+-----------------------------------------------------------------------------------------------------+")
print("Start jet selection.")
for i in range(len(particle.event)):
    for j in range(len(jet.pt[i])):
        if jet.btag[i][j] == 1 and jet.pt[i][j] > 20 and np.abs(jet.eta[i][j]) < 2.4:
            marker_bjet[i][j] = 1 
        elif jet.pt[i][j] > 20 and np.abs(jet.eta[i][j]) <= 2.4:
            marker_jet[i][j] = 1

for i in range(len(particle.event)):
    if np.sum(marker_jet[i] == 1) >= 6 and np.sum(marker_bjet[i] == 1) >= 2 :
        marker_event[i] = 1 
print("Jet selection doen.")
print("+-----------------------------------------------------------------------------------------------------+")

+-----------------------------------------------------------------------------------------------------+
Start jet selection.
Jet selection doen.
+-----------------------------------------------------------------------------------------------------+


In [7]:
jet_pt = []
jet_eta = []
jet_phi = []
jet_mass = []
jet_btag = []

In [8]:
for i in range(len(jet.event)):
    if marker_event[i] == 1:
        jet_pt_tmp = []
        jet_eta_tmp = []
        jet_phi_tmp = []
        jet_mass_tmp = []
        jet_btag_tmp = []
        for j in range(len(jet.pt[i])):
            if marker_jet[i][j] == 1:
                jet_pt_tmp.append(jet.pt[i][j])
                jet_eta_tmp.append(jet.eta[i][j])
                jet_phi_tmp.append(jet.phi[i][j])
                jet_mass_tmp.append(jet.mass[i][j])
                jet_btag_tmp.append(jet.btag[i][j])
                
        jet_pt.append(jet_pt_tmp)
        jet_eta.append(jet_eta_tmp)
        jet_phi.append(jet_phi_tmp)
        jet_mass.append(jet_mass_tmp)
        jet_btag.append(jet_btag_tmp)

In [10]:
len(jet_eta)

331

## Particle tracing and daughter finding section
---

In [11]:
def shift_particle_tracing(dataset, PID_d, idx):
    if (dataset.iloc[idx,6] == PID_d):
        return dataset.iloc[idx,4]

def particle_tracing(dataset, PID, STATUS):

    for i in range(len(dataset)):
        if(dataset.iloc[i,1] == STATUS and dataset.iloc[i,6] == PID ): 
            daughter_index = int(dataset.iloc[i,0])
    if( dataset.iloc[daughter_index,6] == PID ):
        shifted_particle_index = dataset.iloc[daughter_index, 4]


    while dataset.iloc[shifted_particle_index,6] == PID:
            init_shifted_particle_index = shifted_particle_index
            shifted_particle_index = shift_particle_tracing(dataset, PID, init_shifted_particle_index)       

    dauthter_idx_1 = dataset.iloc[init_shifted_particle_index, 4]
    daughter_pid_1 = dataset.iloc[dauthter_idx_1, 6]

    dauthter_idx_2 = dataset.iloc[init_shifted_particle_index, 5]
    daughter_pid_2 = dataset.iloc[dauthter_idx_2, 6]

    return init_shifted_particle_index, dauthter_idx_1, daughter_pid_1, dauthter_idx_2, daughter_pid_2


In [12]:
for i in range(len(particle.event)):
    if marker_event[i] == 1:
        #print("+------------------------------------------------------------------------------------------------------+")
        #print("Start parsing event : {0}\nStart to trace top quark and find its daughters.".format(i))
        top_idx[i], top_daughter_idx_1[i], top_daughter_pid_1[i], top_daughter_idx_2[i], top_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP, 22)
        #print("+------------------------------------------------------~-----------------------------------------------+")
        #print("Start to find top_bar quark and its daughters.")
        top_bar_idx[i], top_bar_daughter_idx_1[i], top_bar_daughter_pid_1[i], top_bar_daughter_idx_2[i], top_bar_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP_BAR, 22)
        #print("+------------------------------------------------------------------------------------------------------+")

### Tracing the daughter 

In [13]:
#Input two daughter of top/top_bar and find their daughter
def quark_finder(dataset, mother_idx_1, mother_idx_2):
    
    #Specific two daughter of top
    def W_b_specifier(dataset, input_1_idx, input_2_idx):
        if dataset.iloc[int(input_1_idx),6] == PID_W_plus or dataset.iloc[int(input_1_idx),6] == PID_W_minus :
            return int(input_1_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_2_idx)
        elif dataset.iloc[int(input_1_idx),6] == PID_BOTTOM or dataset.iloc[int(input_1_idx),6] == PID_BOTTOM_BAR :
            return  int(input_2_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_1_idx)
        else :
            pass
            #print("Please check your data.")
    
    W_boson_idx, mother_pid, b_quark_idx = W_b_specifier(dataset, mother_idx_1, mother_idx_2)
    
    #Find the two daughters of boson
    
    daughter_1_idx = dataset.iloc[W_boson_idx, 4]
    daughter_1_pid = dataset.iloc[daughter_1_idx, 6]
    daughter_2_idx = dataset.iloc[W_boson_idx, 5]
    daughter_2_pid = dataset.iloc[daughter_2_idx, 6]

    
    if daughter_1_pid == mother_pid or daughter_2_pid == mother_pid:

        init_idx = W_boson_idx
        daughter_pid = daughter_1_pid
        if daughter_2_pid == mother_pid:
            daughter_pid = daughter_2_pid
        while daughter_pid == mother_pid :
            daughter_1_idx = dataset.iloc[int(init_idx), 4]
            daughter_2_idx = dataset.iloc[int(init_idx), 5]

            daughter_1_pid = dataset.iloc[int(daughter_1_idx), 6]
            daughter_2_pid = dataset.iloc[int(daughter_2_idx), 6]

            daughter_pid = daughter_1_pid
            init_idx = daughter_1_idx
            if daughter_2_pid == mother_pid:
                daughter_pid = daughter_2_pid
                init_idx = daughter_2_idx
            
            
            #print("Temporary daughter 1 indxe: {0}, PID: {1}".format(daughter_1_idx, daughter_1_pid))
            #print("Temporary daughter 2 indxe: {0}, PID: {1}".format(daughter_2_idx, daughter_2_pid))

    
    #print("Found daughter 1 index: {0}, PID: {1}.\nFound daughter 2 index: {2}, PID: {3}".format(daughter_1_idx, daughter_1_pid, daughter_2_idx, daughter_2_pid))
    return  b_quark_idx, daughter_1_idx, daughter_2_idx

In [14]:
for i in range(len(particle.event)):
    if marker_event[i] == 1 :
        #print("+------------------------------------------------------------------------------------------------------+")
        #print("Start parsing event : {0}\nStart to find top quark's daughters.".format(i))
        parton_array[i][0][0], parton_array[i][1][0], parton_array[i][2][0] = quark_finder(particle.dataframelize(i), top_daughter_idx_1[i], top_daughter_idx_2[i])
        #print("+------------------------------------------------------~-----------------------------------------------+")
        #print("Start to find top_bar quark's daughters.")
        parton_array[i][3][0], parton_array[i][4][0], parton_array[i][5][0], = quark_finder(particle.dataframelize(i), top_bar_daughter_idx_1[i], top_bar_daughter_idx_2[i])
        #print("+------------------------------------------------------------------------------------------------------+")
    elif marker_event[i] == 0 :
        parton_array[i] = 'Nan'
    else: pass

In [15]:
# ### 
# t t~ W+ W- b b~ 
# 0 0  0  0  0 0

# i.e.
# col 3 = 100010 
# col 5 = 101000
# col 6 = 101000
# col 8 = 010100
# col 10= 010001
# col 11= 010001

In [78]:
parton_pdgid = []
parton_barcode = []
parton_pt = []
parton_eta = []
parton_phi = []
parton_mass = []

In [79]:
barcode = np.array([34, 40, 40, 17, 20, 20])
for i in range(len(particle.event)):
    if marker_event[i] == 1:
        _parton_pdgid = []
        _parton_barcode = []
        _parton_pt = []
        _parton_eta = []
        _parton_phi = []
        _parton_mass = []
        for j in range(0,6):
            dataset = particle.dataframelize(i)

            _parton_pdgid.append(dataset.iloc[int(parton_array[i][j][0]), 6])
            _parton_barcode.append(barcode[j])
            _parton_pt.append(dataset.iloc[int(parton_array[i][j][0]), 7])
            _parton_eta.append(dataset.iloc[int(parton_array[i][j][0]), 8])
            _parton_phi.append(dataset.iloc[int(parton_array[i][j][0]), 9])
            _parton_mass.append(dataset.iloc[int(parton_array[i][j][0]), 10])

            # parton_array[i][j][1] = dataset.iloc[int(parton_array[i][j][0]), 6]  #PDGID
            # parton_array[i][j][2] = barcode[j]
            # parton_array[i][j][3] = dataset.iloc[int(parton_array[i][j][0]), 7]  #Pt
            # parton_array[i][j][4] = dataset.iloc[int(parton_array[i][j][0]), 8]  #Eta
            # parton_array[i][j][5] = dataset.iloc[int(parton_array[i][j][0]), 9]  #Phi
            # parton_array[i][j][6] = dataset.iloc[int(parton_array[i][j][0]), 10]  #Mass
        parton_pdgid.append(_parton_pdgid)
        parton_barcode.append(_parton_barcode)
        parton_pt.append(_parton_pt)
        parton_eta.append(_parton_eta)
        parton_phi.append(_parton_phi)
        parton_mass.append(_parton_mass)

In [80]:
parton_pt[6]

[50.585606, 49.724102, 65.15139, 113.06482, 26.74305, 47.041145]

In [81]:
jet_pt[6]

[64.488945, 52.294407, 43.845146, 42.45246, 30.607224, 29.747969]

## Parton-jet matching section
---

### Define the function for computing delta_R


In [82]:
def deltaPhi(phi1,phi2):
    phi = phi1-phi2
    while phi >= np.pi: phi -= np.pi*2.
    while phi < -np.pi: phi += np.pi*2.
    return phi

def delta_R(eta1, phi1, eta2, phi2):
    return np.sqrt(deltaPhi(phi1,phi2)**2+(eta1-eta2)**2)

def min_delta_R(target_1, target_2):
    pass

In [83]:
dR_between_parton_jet = []
dR_between_parton_parton = []

for i in range(len(parton_pdgid)):
    dR_between_parton_jet.append(np.zeros([len(jet_pt[i]) * 6])) # # of connection = num of jet * num of parton
    dR_between_parton_parton.append(np.zeros([15])) # C^{6}_{2} = 15

dR_between_parton_jet = np.asanyarray(dR_between_parton_jet)
dR_between_parton_parton = np.asanyarray(dR_between_parton_parton)



In [84]:
max_num_of_jet_cand = []
for i in range(len(parton_pdgid)):
    max_num_of_jet_cand.append(len(jet_pt[i]))
max_num_of_jet_cand = np.asanyarray(max_num_of_jet_cand)
max_num_of_jet = max_num_of_jet_cand.max()
print(max_num_of_jet)

matching_jet = []
matching_parton = []
for i in range(len(parton_pdgid)):
    matching_jet.append(np.zeros([len(jet_pt[i])]))
    matching_parton.append(np.zeros([6]))

matching_jet = np.array(matching_jet)
matching_parton = np.array(matching_parton)

10


### Computing delta_R between each parton and jet

In [85]:
for i in range(len(dR_between_parton_jet)):
    print(dR_between_parton_jet[i].shape)

(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(48,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(48,)
(42,)
(42,)
(42,)
(36,)
(36,)
(36,)
(36,)
(48,)
(48,)
(36,)
(42,)
(42,)
(42,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(54,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(42,)
(42,)
(42,)
(36,)
(36,)
(36,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(42,)
(36,)
(42,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(48,)
(36,)
(36,)
(42,)
(36,)
(42,)
(42,)
(36,)
(36,)
(42,)
(42,)
(36,)
(42,)
(36,)
(42,)
(36,)
(36,)
(36,)
(42,)
(36,)
(42,)
(36,)
(36,)
(42,)
(36,)
(36,)
(42,)
(36,)
(42,)
(42,)
(48,)
(54,)
(48,)
(36,)
(36,)
(36,)
(36,)
(36,)
(42,)
(36,)
(42,)
(36,)
(36,)
(42,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(42,)
(42,)
(36,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(48,)
(42,)
(36,)
(36,)
(42,)
(42,)
(36,)
(36,)
(36,)
(42,)
(36,)
(36,)
(42,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(36,)
(42,)
(36,)
(36,)
(36,

In [86]:
parton_eta[6]

[-0.15873247, 0.6501244, 0.72577214, 0.073509455, 0.19216959, -1.168629]

In [87]:
parton_phi[6]

[-2.245016, 1.3704467, -0.05532509, -1.5512011, 1.9859097, -2.214733]

In [88]:
jet_eta[6]

[0.71157384, 0.6531161, -1.136138, -0.30443662, 1.8715075, 0.2948797]

In [89]:
jet_phi[6]

[-0.09117323, 1.3331826, -2.18164, 2.7710462, 0.31985694, 2.1293974]

In [90]:
dR_between_parton_jet[6]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [91]:
print(b)

5


In [92]:
for i in range(len(parton_pdgid)):
    j = 0
    a = 0
    b = 0
    while a < 6 :
        for b in range( len(jet_pt[i]) ):
            print(i, j)
            #print(i, a, b)
            #print(delta_R( parton_array[i][a][4], parton_array[i][a][5], jet.eta[i][b], jet.phi[i][b]))
            dR_between_parton_jet[i][j] = delta_R( parton_eta[i][a], parton_phi[i][a], jet_eta[i][b], jet_phi[i][b])
            j +=1
        a += 1 


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
4 31
4 32
4 33
4 34
4 35
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25


### Matching jet and parton by finding the Min(dR(parton, jet))

In [93]:

for i in range(len(parton_pt)):
    
    #print("+------------------------------------------------------------------------------------------------------+")
    #print(dR_between_parton_jet.shape)
    array = np.reshape(dR_between_parton_jet[i], [6, len(jet_pt[i])])
    #print(array.shape)
        
    dataset = pd.DataFrame({'0': array[0,:], 
                                '1': array[1,:],
                                '2': array[2,:],
                                '3': array[3,:],
                                '4': array[4,:],
                                '5': array[5,:],
                                })
    #print(dataset)

    for j in range(0,6):
        #print("+------------------------------------------------------------------------------------------------------+")
        min_val = dataset.stack().min()
        if min_val < 0.4:
            #print("Min val: {0}".format(min_val))
            min_idx, min_col = dataset.stack().idxmin()
            matching_parton[i][j] = int(min_idx)
            matching_jet[i][j] = int(min_col)
            #print("The position of minimun appears. Raw: {0}, Colume: {1}".format(min_idx, min_col))
            dataset = dataset.drop([min_col], axis=1)
            dataset = dataset.drop([min_idx], axis=0)
            #print("The dataset after delete the minimun's raw and colume:")
            #print(dataset)
        else:
            matching_parton[i][j] = 'Nan'
            matching_jet[i][j] = 'Nan'
    for k in range(6, len(jet_pt[i])):
        matching_jet[i][k] = 'Nan'

        

In [94]:
for i in range(len(parton_pdgid)):
    print(i)
    array = np.reshape(dR_between_parton_jet[i], [6, len(jet_pt[i])])
    print(matching_parton[i], matching_jet[i])

0
[ 2.  3.  4. nan nan nan] [ 2.  1.  4. nan nan nan]
1
[ 1.  4.  3.  6. nan nan] [ 5.  4.  1.  2. nan nan nan]
2
[ 2.  1.  3.  4. nan nan] [ 4.  1.  2.  5. nan nan]
3
[ 1.  0.  4.  3. nan nan] [ 4.  1.  2.  0. nan nan]
4
[ 3.  2.  1. nan nan nan] [ 2.  4.  1. nan nan nan]
5
[ 0.  3.  2. nan nan nan] [ 3.  2.  4. nan nan nan]
6
[ 1.  0.  2.  5. nan nan] [ 1.  2.  5.  4. nan nan]
7
[ 6.  4.  2.  5.  1. nan] [ 4.  5.  1.  2.  0. nan nan]
8
[ 0.  2.  1. nan nan nan] [ 5.  1.  2. nan nan nan]
9
[ 2.  3.  1.  4. nan nan] [ 1.  4.  5.  2. nan nan]
10
[ 0.  5.  1.  4. nan nan] [ 2.  5.  1.  4. nan nan]
11
[ 3.  2.  1.  4. nan nan] [ 2.  1.  5.  4. nan nan]
12
[ 0. nan nan nan nan nan] [ 1. nan nan nan nan nan]
13
[ 2.  3.  1.  5.  4. nan] [ 5.  2.  0.  1.  4. nan]
14
[ 5.  1.  2. nan nan nan] [ 5.  2.  4. nan nan nan]
15
[ 1.  2.  3. nan nan nan] [ 3.  2.  5. nan nan nan nan nan]
16
[ 1.  2.  3. nan nan nan] [ 5.  2.  4. nan nan nan]
17
[ 3.  1. nan nan nan nan] [ 1.  2. nan nan nan nan nan]


In [95]:
matching_parton[8]

array([ 0.,  2.,  1., nan, nan, nan])

In [96]:
matching_jet[8]

array([ 5.,  1.,  2., nan, nan, nan])

In [97]:
array = np.reshape(dR_between_parton_jet[8], [6, len(jet_pt[8])])
print(array[3,1])
dataset = pd.DataFrame({'0': array[0,:], 
                                '1': array[1,:],
                                '2': array[2,:],
                                '3': array[3,:],
                                '4': array[4,:],
                                '5': array[5,:],
                                })
print(dataset)
min_idx, min_col = dataset.stack().idxmin()
print(min_idx, min_col)
print(dataset.iloc[4, 5])

2.8501988986756905
          0         1         2         3         4         5
0  2.527664  1.550602  2.080218  1.899760  1.893364  0.046563
1  1.564013  1.640409  0.180967  2.850199  1.096430  2.198584
2  2.801638  0.125186  1.368966  2.824661  0.580109  1.437385
3  1.882490  1.198073  0.273169  2.973053  0.687879  1.875009
4  3.506713  4.839982  4.298667  4.529254  4.844028  3.544244
5  1.600575  3.969284  2.907043  2.769274  3.664835  2.463019
0 5
3.5442439026860812


In [98]:
parton_jet_index = np.zeros([len(parton_pdgid), 6])
jet_parton_index = []
np.zeros([len(parton_pdgid), 6])
for i in range(len(parton_pdgid)):
    jet_parton_index.append(np.zeros([len(jet_pt[i])]))


for i in range(len(parton_pdgid)):
    for j in range(0,6):
        parton_jet_index[i][j] = matching_parton[i][j]
    for k in range(len(jet_pt[i])):
        jet_parton_index[i][k] = matching_jet[i][k]

In [99]:
jet_barcode = []
for i in range(len(parton_pdgid)):
    jet_barcode.append(np.zeros([len(jet_pt[i])]))

jet_barcode = np.array(jet_barcode)

for i in range(len(parton_pdgid)):
    for j in range(len(jet_parton_index[i])):
        if jet_parton_index[i][j] == 0:
            jet_barcode[i][j] = barcode[0]
        elif jet_parton_index[i][j] == 1: 
            jet_barcode[i][j] = barcode[1]
        elif jet_parton_index[i][j] == 2: 
            jet_barcode[i][j] = barcode[2]
        elif jet_parton_index[i][j] == 3: 
            jet_barcode[i][j] = barcode[3]
        elif jet_parton_index[i][j] == 4: 
            jet_barcode[i][j] = barcode[4]
        elif jet_parton_index[i][j] == 5: 
            jet_barcode[i][j] = barcode[5]
        else :
            jet_barcode[i][j] = 'Nan'


In [100]:
jet_barcode[8]

array([20., 40., 40., nan, nan, nan])

## Saved selected events
---

In [101]:
lene = len(parton_pdgid)
#Save the event which pass the selection
with h5py.File("./event_record_pptt_1k.h5",'w') as f:
    dt = h5py.vlen_dtype(np.dtype('int32'))

    hdf5_jet_parton_index = f.create_dataset('jet_parton_index', (lene, ), dtype=dt)
    hdf5_jet_barcode = f.create_dataset('jet_barcode', (lene, ), dtype=dt)
    hdf5_jet_pt = f.create_dataset('jet_pt', (lene, ), dtype=dt)
    hdf5_jet_eta = f.create_dataset('jet_eta', (lene, ), dtype=dt)
    hdf5_jet_phi = f.create_dataset('jet_phi', (lene, ), dtype=dt)
    hdf5_jet_mass = f.create_dataset('jet_mass', (lene, ), dtype=dt)
    hdf5_jet_btag = f.create_dataset('jet_btag', (lene, ), dtype=dt)

    for i in range(lene):
        hdf5_jet_parton_index[i] = jet_parton_index[i]
        hdf5_jet_barcode[i] = jet_barcode[i]
        hdf5_jet_pt[i] = jet_pt[i]
        hdf5_jet_eta[i] = jet_eta[i]
        hdf5_jet_phi[i] = jet_phi[i]
        hdf5_jet_mass[i] = jet_mass[i]
        hdf5_jet_btag[i] = jet_btag[i]

    hdf5_parton_jet_index = f.create_dataset('hdf5_parton_jet_index', (lene, ), dtype=dt)
    hdf5_parton_pdgid = f.create_dataset('hdf5_parton_pdgid', (lene, ), dtype=dt)
    hdf5_parton_barcode = f.create_dataset('hdf5_parton_barcode', (lene, ), dtype=dt)
    hdf5_parton_pt = f.create_dataset('hdf5_parton_pt', (lene, ), dtype=dt)
    hdf5_parton_eta = f.create_dataset('hdf5_parton_eta', (lene, ), dtype=dt)
    hdf5_parton_phi = f.create_dataset('hdf5_parton_phi', (lene, ), dtype=dt)
    hdf5_parton_mass = f.create_dataset('hdf5_parton_mass', (lene, ), dtype=dt)

    for i in range(lene):
        hdf5_parton_jet_index[i] = parton_jet_index[i]
        hdf5_parton_pdgid[i] = parton_pdgid[i]
        hdf5_parton_barcode[i] = parton_barcode[i]
        hdf5_parton_pt[i] = parton_pt[i]
        hdf5_parton_eta[i] = parton_eta[i]
        hdf5_parton_phi[i] = parton_phi[i]
        hdf5_parton_mass[i] = parton_mass[i]
    

In [102]:
# with h5py.File("./hdf5_files/event_record_3.h5",'r') as f:
#     print(f.keys())
#     jet__pt = f['jet_pt']
#     print(len(jet__pt))